<a href="https://colab.research.google.com/github/martha-rincon-lopez/Diplomado/blob/Tareas/Modelos_de_regresi%C3%B3n_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Classification**

#Librerias

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

#Data Loading

In [ ]:
iris = datasets.load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['Species'] = iris.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

#Exploratory Analysis

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   Species            150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [ ]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   Species            150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [ ]:
class_counts = df['Species'].value_counts()
class_names = iris.target_names

print("Class Distribution: ")
for i, count in enumerate(class_counts):
    class_name = class_names[i]
    percentage = (count / len(df)) * 100
    print(f"{class_name}: {count} ({percentage:.2f}%)")

Class Distribution: 
setosa: 50 (33.33%)
versicolor: 50 (33.33%)
virginica: 50 (33.33%)


In [ ]:
fig = px.bar(x = class_counts.index, y = class_counts.values, labels = {'x': 'Class', 'y': 'Count'}, title ='Target Class Distribution', template = 'plotly_white', text = class_counts.values)
fig.show()

In [ ]:
class_0 = df[df['Species'] == 0]
class_1 = df[df['Species'] == 1]
class_2 = df[df['Species'] == 2]

features = iris.feature_names

print("\nFeature Distributions by Class:")
for feature in features:
    fig = px.histogram(df, x = feature, color = 'Species', barmode ='overlay', title = f"{feature} Distribution by Class")
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.80)
    fig.show()


Feature Distributions by Class:


In [ ]:
corr_matrix = df.corr()

fig = px.imshow(corr_matrix, color_continuous_scale = 'RdBu', range_color=[-1,1], text_auto = True, aspect = "auto", labels = dict(color = "Correlation"), title ='Feature Correlation', height = 800)
fig.update(layout_coloraxis_showscale = False)
fig.show()

# Classification Models

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df['Species'], test_size = 0.2, random_state = 7)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Confusion Matrix

In [ ]:
def plot_confusion_matrix(cm, labels):
    fig_cm = px.imshow(cm, labels = dict(x = "Predicted", y = "Actual", color ="Count"),
                       x = labels, y = labels, color_continuous_scale = 'Viridis', text_auto = True,
                       title = "Confusion Matrix")
    fig_cm.update_layout(coloraxis_showscale = False)
    fig_cm.show()

#Logistic Regression

#Train Model

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)

LogisticRegression()

#Evaluate Model

In [ ]:
lr_pred = lr.predict(X_test_scaled)

In [ ]:
lr_accuracy = accuracy_score(y_test, lr_pred)
print(f"Logistic Regression Accuracy: {lr_accuracy:.2f}")

Logistic Regression Accuracy: 0.87


In [ ]:
lr_precision = precision_score(y_test, lr_pred, average = "micro")
print(f"Logistic Regression Precission: {lr_precision:.2f}")

Logistic Regression Precission: 0.87


In [ ]:
lr_recall = recall_score(y_test, lr_pred, average = "micro")
print(f"Logistic Regression Recall: {lr_recall:.2f}")

Logistic Regression Recall: 0.87


In [ ]:
lr_f1 = f1_score(y_test, lr_pred, average = "micro")
print(f"Logistic Regression F1 Score: {lr_f1:.2f}")

Logistic Regression F1 Score: 0.87


In [ ]:
lr_report = classification_report(y_test, lr_pred)
print("Logistic Regression Classification Report:")
print(lr_report)

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       0.83      0.83      0.83        12
           2       0.82      0.82      0.82        11

    accuracy                           0.87        30
   macro avg       0.88      0.88      0.88        30
weighted avg       0.87      0.87      0.87        30



In [ ]:
lr_cm = confusion_matrix(y_test, lr_pred)
plot_confusion_matrix(lr_cm, ['setosa', 'versicolor', 'virginica'])

# K-Nearest Neighbors (KNN)

#Train Model

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

#Evaluate Model

In [ ]:
knn_pred = knn.predict(X_test_scaled)

In [ ]:
knn_accuracy = accuracy_score(y_test, knn_pred)
print(f"KNN Accuracy: {knn_accuracy:.2f}")

KNN Accuracy: 0.87


In [ ]:
knn_precision = precision_score(y_test, knn_pred, average = "micro")
print(f"KNN Precission: {knn_precision:.2f}")

KNN Precission: 0.87


In [ ]:
knn_recall = recall_score(y_test, knn_pred, average = "micro")
print(f"KNN Recall: {knn_recall:.2f}")

KNN Recall: 0.87


In [ ]:
knn_f1 = f1_score(y_test, knn_pred, average = "micro")
print(f"KNN F1 Score: {knn_f1:.2f}")

KNN F1 Score: 0.87


In [ ]:
knn_report = classification_report(y_test, knn_pred)
print("KNN Classification Report:")
print(knn_report)

KNN Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.86      0.92         7
           1       0.75      1.00      0.86        12
           2       1.00      0.73      0.84        11

    accuracy                           0.87        30
   macro avg       0.92      0.86      0.87        30
weighted avg       0.90      0.87      0.87        30



In [ ]:
knn_cm = confusion_matrix(y_test, knn_pred)
plot_confusion_matrix(knn_cm, ['setosa', 'versicolor', 'virginica'])

# Support Vector Machines (SVM)

#Train Model

In [ ]:
svm = SVC()
svm.fit(X_train_scaled, y_train)

SVC()

#Evaluate Model

In [ ]:
svm_pred = svm.predict(X_test_scaled)

In [ ]:
svm_accuracy = accuracy_score(y_test, svm_pred)
print(f"SVM Accuracy: {svm_accuracy:.2f}")

SVM Accuracy: 0.87


In [ ]:
svm_precision = precision_score(y_test, svm_pred, average = "micro")
print(f"SVM Precission: {svm_precision:.2f}")

SVM Precission: 0.87


In [ ]:
svm_recall = recall_score(y_test, svm_pred, average = 'micro')
print(f"KNN Recall: {svm_recall:.2f}")

KNN Recall: 0.87


In [ ]:
svm_f1 = f1_score(y_test, svm_pred, average = 'micro')
print(f"SVM F1 Score: {svm_f1:.2f}")

SVM F1 Score: 0.87


In [ ]:
svm_report = classification_report(y_test, svm_pred)
print("SVM Classification Report:")
print(svm_report)

SVM Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       0.83      0.83      0.83        12
           2       0.82      0.82      0.82        11

    accuracy                           0.87        30
   macro avg       0.88      0.88      0.88        30
weighted avg       0.87      0.87      0.87        30



In [ ]:
svm_cm = confusion_matrix(y_test, svm_pred)
plot_confusion_matrix(svm_cm, ['setosa', 'versicolor', 'virginica'])

# Naive Bayes

#Train Model

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)

GaussianNB()

#Evaluate Model

In [ ]:
nb_pred = nb.predict(X_test)

In [ ]:
nb_accuracy = accuracy_score(y_test, nb_pred)
print(f"Naive Bayes Accuracy: {nb_accuracy:.2f}")

Naive Bayes Accuracy: 0.83


In [ ]:
nb_precision = precision_score(y_test, nb_pred, average = 'micro')
print(f"Naive Bayes Precission: {nb_precision:.2f}")

Naive Bayes Precission: 0.83


In [ ]:
nb_recall = recall_score(y_test, nb_pred, average = 'micro')
print(f"Naive Bayes Recall: {nb_recall:.2f}")

Naive Bayes Recall: 0.83


In [ ]:
nb_f1 = f1_score(y_test, nb_pred, average = 'micro')
print(f"Naive Bayes F1 Score: {nb_f1:.2f}")

Naive Bayes F1 Score: 0.83


In [ ]:
nb_report = classification_report(y_test, nb_pred)
print("Naive Bayes Classification Report:")
print(nb_report)

Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       0.82      0.75      0.78        12
           2       0.75      0.82      0.78        11

    accuracy                           0.83        30
   macro avg       0.86      0.86      0.86        30
weighted avg       0.84      0.83      0.83        30



In [ ]:
nb_cm = confusion_matrix(y_test, nb_pred)
plot_confusion_matrix(nb_cm, ['setosa', 'versicolor', 'virginica'])

# Decision Trees

#Train Model

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier()

#Evaluate Model

In [ ]:
dt_pred = dt.predict(X_test)

In [ ]:
dt_accuracy = accuracy_score(y_test, dt_pred)
print(f"Decision Trees Accuracy: {dt_accuracy:.2f}")

Decision Trees Accuracy: 0.90


In [ ]:
dt_precision = precision_score(y_test, dt_pred, average = 'micro')
print(f"Decision Trees Precission: {dt_precision:.2f}")

Decision Trees Precission: 0.90


In [ ]:
dt_recall = recall_score(y_test, dt_pred, average = 'micro')
print(f"Decision Trees Recall: {dt_recall:.2f}")

Decision Trees Recall: 0.90


In [ ]:
dt_f1 = f1_score(y_test, dt_pred, average = 'micro')
print(f"Decision Trees F1 Score: {dt_f1:.2f}")

Decision Trees F1 Score: 0.90


In [ ]:
dt_report = classification_report(y_test, dt_pred)
print("Decision Trees Classification Report:")
print(dt_report)

Decision Trees Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       0.91      0.83      0.87        12
           2       0.83      0.91      0.87        11

    accuracy                           0.90        30
   macro avg       0.91      0.91      0.91        30
weighted avg       0.90      0.90      0.90        30



In [ ]:
dt_cm = confusion_matrix(y_test, dt_pred)
plot_confusion_matrix(dt_cm, ['setosa', 'versicolor', 'virginica'])

# Random Forest

#Train Model

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

#Evaluate Model

In [ ]:
rf_pred = rf.predict(X_test)

In [ ]:
rf_accuracy = accuracy_score(y_test, rf_pred)
print(f"Random Forest Accuracy: {rf_accuracy:.2f}")

Random Forest Accuracy: 0.87


In [ ]:
rf_precision = precision_score(y_test, rf_pred, average = 'micro')
print(f"Random Forest Precission: {rf_precision:.2f}")

Random Forest Precission: 0.87


In [ ]:
rf_recall = recall_score(y_test, rf_pred, average = 'micro')
print(f"Random Forest Recall: {rf_recall:.2f}")

Random Forest Recall: 0.87


In [ ]:
rf_f1 = f1_score(y_test, rf_pred, average = 'micro')
print(f"Random Forest F1 Score: {rf_f1:.2f}")

Random Forest F1 Score: 0.87


In [ ]:
rf_report = classification_report(y_test, rf_pred)
print("Random Forest Classification Report:")
print(rf_report)

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       0.83      0.83      0.83        12
           2       0.82      0.82      0.82        11

    accuracy                           0.87        30
   macro avg       0.88      0.88      0.88        30
weighted avg       0.87      0.87      0.87        30



In [ ]:
rf_cm = confusion_matrix(y_test, rf_pred)
plot_confusion_matrix(rf_cm, ['setosa', 'versicolor', 'virginica'])

# Comparing Models

In [ ]:
models_data = {
    'Model': ['Logistic Regression', 'K-Nearest Neighbors', 'SVM', 'Naive Bayes', 'Decision Trees', 'Random Forest'],
    'Accuracy': [lr_accuracy, knn_accuracy, svm_accuracy, nb_accuracy, dt_accuracy, rf_accuracy],
    'Precision': [lr_precision, knn_precision, svm_precision, nb_precision, dt_precision, rf_precision],
    'Recall': [lr_recall, knn_recall, svm_recall, nb_recall, dt_recall, rf_recall],
    'F1-Score': [lr_f1, knn_f1, svm_f1, nb_f1, dt_f1, rf_f1]
}

df_models = pd.DataFrame(models_data)
df_models

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.866667,0.866667,0.866667,0.866667
1,K-Nearest Neighbors,0.866667,0.866667,0.866667,0.866667
2,SVM,0.866667,0.866667,0.866667,0.866667
3,Naive Bayes,0.833333,0.833333,0.833333,0.833333
4,Decision Trees,0.900000,0.900000,0.900000,0.900000
5,Random Forest,0.866667,0.866667,0.866667,0.866667


In [ ]:
fig_accuracy = px.bar(df_models, x ='Model', y ='Accuracy', title = 'Model Comparison - Accuracy', template = 'plotly_white', text = 'Accuracy')
fig_accuracy.update_layout(yaxis = dict(range = [0, 1]))
fig_accuracy.show()

In [ ]:
fig_precision = px.bar(df_models, x = 'Model', y ='Precision', title ='Model Comparison - Precision', template = 'plotly_white', text = 'Precision')
fig_precision.update_layout(yaxis=dict(range = [0, 1]))
fig_precision.show()

In [ ]:
fig_recall = px.bar(df_models, x ='Model', y ='Recall', title ='Model Comparison - Recall', template = 'plotly_white', text = 'Recall')
fig_recall.update_layout(yaxis=dict(range = [0, 1]))
fig_recall.show()

In [ ]:
fig_f1_score = px.bar(df_models, x ='Model', y='F1-Score', title ='Model Comparison - F1-Score', template = 'plotly_white', text = 'Recall')
fig_f1_score.update_layout(yaxis=dict(range = [0, 1]))
fig_f1_score.show()